In [2]:
import numpy as np
from collections import OrderedDict
from transformers import MPNetPreTrainedModel, MPNetModel, AutoTokenizer
import torch

In [21]:
# Mean Pooling - Take attention mask into account for correct averaging
def mean_pooling(model_output, attention_mask):
        token_embeddings = model_output #First element of model_output contains all token embeddings
        input_mask_expanded = attention_mask.unsqueeze(-1).expand(token_embeddings.size()).float()
        return torch.sum(token_embeddings * input_mask_expanded, 1) / torch.clamp(input_mask_expanded.sum(1), min=1e-9)

# Definition of ESGify class because of custom,sentence-transformers like, mean pooling function and classifier head
class ESGify(MPNetPreTrainedModel):
    """Model for Classification ESG risks from text."""

    def __init__(self,config): #tuning only the head
        super().__init__(config)
        # Instantiate Parts of model
        self.mpnet = MPNetModel(config,add_pooling_layer=False)
        self.id2label =  config.id2label
        self.label2id =  config.label2id
        self.classifier = torch.nn.Sequential(OrderedDict([('norm',torch.nn.BatchNorm1d(768)),
                                                ('linear',torch.nn.Linear(768,512)),
                                                ('act',torch.nn.ReLU()),
                                                ('batch_n',torch.nn.BatchNorm1d(512)),
                                                ('drop_class', torch.nn.Dropout(0.2)),
                                                ('class_l',torch.nn.Linear(512 ,47))]))


    def forward(self, input_ids, attention_mask):
         # Feed input to mpnet model
        outputs = self.mpnet(input_ids=input_ids,
                             attention_mask=attention_mask)
         
        # mean pooling dataset and eed input to classifier to compute logits
        logits = self.classifier( mean_pooling(outputs['last_hidden_state'],attention_mask))
         
        # apply sigmoid
        logits  = 1.0 / (1.0 + torch.exp(-logits))
        return logits

model = ESGify.from_pretrained('ai-lab/ESGify')
tokenizer = AutoTokenizer.from_pretrained('ai-lab/ESGify')

texts = ["Шольц может приехать в Москву до досрочных парламентских выборов, которые пройдут в Германии 23 февраля, заявил Кизеветтер. Кремль опровергал подготовку визита немецкого канцлера. Канцлер Германии Олаф Шольц может посетить Москву до досрочных парламентских выборов, назначенных в стране на 23 февраля. Об этом заявил депутат бундестага от Христианско-демократического союза (ХДС) Родерих Кизеветтер в социальной сети X.«Встреча [будущего президента США Дональда] Трампа и Путина в настоящее время запланирована на март», — добавил парламентарий, не уточнив источник этих сведений. В Германии 23 февраля пройдут досрочные парламентские выборы. Президент страны Франк-Вальтер Штайнмайер распустил бундестаг и назначил их после того, как парламент отказал в доверии правительству Шольца. В ноябре в Германии распалась правящая «светофорная» коалиция, куда входили Социал-демократическая партия канцлера (СДПГ), Свободная демократическая партия (СвДП) и «Зеленые»."]
to_model = tokenizer.batch_encode_plus(
                  texts,
                  add_special_tokens=True,
                  max_length=140,
                  return_token_type_ids=False,
                  padding="max_length",
                  truncation=True,
                  return_attention_mask=True,
                  return_tensors='pt',
                )
results = model(**to_model)
ban = ['Economic Crime', 'Legal Proceedings & Law Violations', 'Corporate Governance', 'Values and Ethics', 'Risk Management and Internal Control', 'Strategy Implementation', 'Disclosure', 'Responsible Investment & Greenwashing', 'Supply Chain (Economic / Governance)', 'Indigenous People', 'Human Rights', 'Emergencies (Social)', 'Land Acquisition and Resettlement (S)', 'Data Safety', 'Freedom of Association and Right to Organise', 'Minimum Age and Child Labour', 'Forced Labour', 'Discrimination', 'Retrenchment', 'Labor Relations Management', 'Supply Chain (Social)']

if str(model.id2label[torch.topk(results, k=47).indices.tolist()[0][0]]) not in ban:
    print("in ESGFY")
else:
    print("not in ESGFY")

in ESGFY


In [23]:
import re
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM

WHITESPACE_HANDLER = lambda k: re.sub('\s+', ' ', re.sub('\n+', ' ', k.strip()))

article_text = texts[0]

model_name = "csebuetnlp/mT5_multilingual_XLSum"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForSeq2SeqLM.from_pretrained(model_name)

input_ids = tokenizer(
    [WHITESPACE_HANDLER(article_text)],
    return_tensors="pt",
    padding="max_length",
    truncation=True,
    max_length=len(article_text)
)["input_ids"]

output_ids = model.generate(
    input_ids=input_ids,
    max_length=50,
    no_repeat_ngram_size=2,
    num_beams=4
)[0]

summary = tokenizer.decode(
    output_ids,
    skip_special_tokens=True,
    clean_up_tokenization_spaces=False
)

print(summary)


tokenizer_config.json:   0%|          | 0.00/375 [00:00<?, ?B/s]

C:\Users\ЯРОСЛАВ\AppData\Local\Programs\Python\Python311\Lib\site-packages\huggingface_hub\file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


config.json:   0%|          | 0.00/730 [00:00<?, ?B/s]

spiece.model:   0%|          | 0.00/4.31M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/65.0 [00:00<?, ?B/s]

You are using the default legacy behaviour of the <class 'transformers.models.t5.tokenization_t5.T5Tokenizer'>. This is expected, and simply means that the `legacy` (previous) behavior will be used so nothing changes for you. If you want to use the new behaviour, set `legacy=False`. This should only be set if you understand what it means, and thoroughly read the reason why this was added as explained in https://github.com/huggingface/transformers/pull/24565
C:\Users\ЯРОСЛАВ\AppData\Local\Programs\Python\Python311\Lib\site-packages\transformers\convert_slow_tokenizer.py:560: UserWarning: The sentencepiece tokenizer that you are converting to a fast tokenizer uses the byte fallback option which is not implemented in the fast tokenizers. In practice this means that the fast version of the tokenizer can produce unknown tokens whereas the sentencepiece version would have converted these unknown tokens into a sequence of byte tokens matching the original piece of text.
  warnings.warn(


pytorch_model.bin:   0%|          | 0.00/2.33G [00:00<?, ?B/s]

Error while downloading from https://cdn-lfs.hf.co/csebuetnlp/mT5_multilingual_XLSum/1899a041aceedfd0c9c67e87f2597bc597ce6f4c1f21b5d35a6325322608a898?response-content-disposition=inline%3B+filename*%3DUTF-8%27%27pytorch_model.bin%3B+filename%3D%22pytorch_model.bin%22%3B&response-content-type=application%2Foctet-stream&Expires=1739465713&Policy=eyJTdGF0ZW1lbnQiOlt7IkNvbmRpdGlvbiI6eyJEYXRlTGVzc1RoYW4iOnsiQVdTOkVwb2NoVGltZSI6MTczOTQ2NTcxM319LCJSZXNvdXJjZSI6Imh0dHBzOi8vY2RuLWxmcy5oZi5jby9jc2VidWV0bmxwL21UNV9tdWx0aWxpbmd1YWxfWExTdW0vMTg5OWEwNDFhY2VlZGZkMGM5YzY3ZTg3ZjI1OTdiYzU5N2NlNmY0YzFmMjFiNWQzNWE2MzI1MzIyNjA4YTg5OD9yZXNwb25zZS1jb250ZW50LWRpc3Bvc2l0aW9uPSomcmVzcG9uc2UtY29udGVudC10eXBlPSoifV19&Signature=TCi9t-Mm5SDWZsv1wYe0dS93c9Jwj6UftW88lSEG7OvzDGmIFlbYFh7fbLn1LuUvVAItURxdy7CI03bnBbP7MLfw1jkuQazuvLjrjOFtwzvuzac%7Ey%7EH1Zr0ljjmuJk7f7AMZ2cwVSPZbVFD974rwhTSfTgIc0srBOPfWfXsuU8eOsJe4CBYn5Cvc69k7hAqv9xvphmtuHRDj5Q3yk6aPE39gJ13q5GjFDYK5a2TJyGUV6QYh5TsBovtoxYK4ovUJn%7EffEpVQ7YFg350il6-4TMdzIMH2z

pytorch_model.bin:  23%|##3       | 545M/2.33G [00:00<?, ?B/s]

Канцлер Германии Олаф Шольц может посетить Москву на март, заявил депутат бундестага Родерих Кизеветтер.
